Лаборатона робота №2

Виконав студент групи ФБ-35 Куб'юк Дмитро

Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження.

In [16]:
import urllib.request
import datetime
import os
import pandas as pd

In [17]:
os.makedirs("downloads")

def download(n):
    time = datetime.datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={n}&year1=1981&year2=2025&type=Mean"
    vhi_url = urllib.request.urlopen(url)
    
    out_path = os.path.join("downloads", f"vhi_id_{n}_{time}.csv")
    with open(out_path, 'wb') as out:
        out.write(vhi_url.read()) 
    print(f"VHI for provinceID={n} is downloaded to {out_path}...")

for i in range(1, 28):
    download(i)


VHI for provinceID=1 is downloaded to downloads\vhi_id_1_11-03-2025_11-49-10.csv...
VHI for provinceID=2 is downloaded to downloads\vhi_id_2_11-03-2025_11-49-11.csv...
VHI for provinceID=3 is downloaded to downloads\vhi_id_3_11-03-2025_11-49-12.csv...
VHI for provinceID=4 is downloaded to downloads\vhi_id_4_11-03-2025_11-49-13.csv...
VHI for provinceID=5 is downloaded to downloads\vhi_id_5_11-03-2025_11-49-14.csv...
VHI for provinceID=6 is downloaded to downloads\vhi_id_6_11-03-2025_11-49-15.csv...
VHI for provinceID=7 is downloaded to downloads\vhi_id_7_11-03-2025_11-49-15.csv...
VHI for provinceID=8 is downloaded to downloads\vhi_id_8_11-03-2025_11-49-16.csv...
VHI for provinceID=9 is downloaded to downloads\vhi_id_9_11-03-2025_11-49-17.csv...
VHI for provinceID=10 is downloaded to downloads\vhi_id_10_11-03-2025_11-49-18.csv...
VHI for provinceID=11 is downloaded to downloads\vhi_id_11_11-03-2025_11-49-19.csv...
VHI for provinceID=12 is downloaded to downloads\vhi_id_12_11-03-2025_11

Зчитати завантажені текстові файли у фрейм. Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [18]:
def read(dir):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    files = os.listdir(dir)
    dataframe = pd.DataFrame()

    for file in files:
        path = os.path.join(dir, file)
        df = pd.read_csv(path, header = 1, names = headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop(columns=['empty'])
        df = df.dropna()
        df['area'] = file.split("_")[2]
        df['Year'] = df['Year'].str.replace('<tt><pre>', '').str.replace('</pre></tt>', '')
        dataframe = pd.concat([dataframe, df], ignore_index=True).drop_duplicates()
    return dataframe

dataframe = read("downloads/")
print(dataframe)


       Year  Week    SMN     SMT    VCI    TCI    VHI area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95   10
1      1982   2.0  0.063  261.53  55.89  38.20  47.04   10
2      1982   3.0  0.063  263.45  57.30  32.69  44.99   10
3      1982   4.0  0.061  265.10  53.96  28.62  41.29   10
4      1982   5.0  0.058  266.42  46.87  28.57  37.72   10
...     ...   ...    ...     ...    ...    ...    ...  ...
59260  2025   5.0  0.120  275.94  71.30   5.61  38.45    9
59261  2025   6.0  0.127  276.50  70.87   8.66  39.76    9
59262  2025   7.0  0.134  277.28  69.12  12.84  40.98    9
59263  2025   8.0  0.142  277.91  66.11  18.50  42.30    9
59264  2025   9.0  0.152  278.99  64.03  23.10  43.57    9

[59265 rows x 8 columns]


Реалізувати окрему процедуру, яка змінить індекси областей, які використані на порталі NOAA (за англійською абеткою) на наступні, за українською (виключно старі індекси на нові):

In [19]:
old_indexes = {
    1: "Черкаська", 2: "Чернігівська", 3: "Чернівецька", 4: "Республіка Крим",
    5: "Дніпропетровська", 6: "Донецька", 7: "Івано-Франківська", 8: "Харківська",
    9: "Херсонська", 10: "Хмельницька", 11: "Київська", 12: "Київська", 13: "Кіровоградська",
    14: "Луганська", 15: "Львівська", 16: "Миколаївська", 17: "Одеська", 18: "Полтавська",
    19: "Рівненська", 20: "Севастополь", 21: "Сумська", 22: "Тернопільська", 23: "Закарпатська",
    24: "Вінницька", 25: "Волинська", 26: "Запорізька", 27: "Житомирська"
}

new_indexes = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька",
    5: "Житомирська", 6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська",
    9: "Київська", 10: "Кіровоградська", 11: "Луганська", 12: "Львівська",
    13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 16: "Рівненська",
    17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська",
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська",
    25: "Республіка Крим"
}

def replace_indexes(dataframe):
    dataframe['area'] = dataframe['area'].astype(int)
    dataframe = dataframe[dataframe['area'] != 11]
    dataframe = dataframe[dataframe['area'] != 20]

    dataframe['area'] = dataframe['area'].replace(old_indexes)

    reverse_indexes = dict(map(lambda x: (x[1], x[0]), new_indexes.items()))
    dataframe['area'] = dataframe['area'].replace(reverse_indexes)
    
    dataframe['area'] = dataframe['area'].astype(int)
    return dataframe

dataframe = replace_indexes(dataframe)
print(dataframe)



       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    21
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
...     ...   ...    ...     ...    ...    ...    ...   ...
59260  2025   5.0  0.120  275.94  71.30   5.61  38.45    20
59261  2025   6.0  0.127  276.50  70.87   8.66  39.76    20
59262  2025   7.0  0.134  277.28  69.12  12.84  40.98    20
59263  2025   8.0  0.142  277.91  66.11  18.50  42.30    20
59264  2025   9.0  0.152  278.99  64.03  23.10  43.57    20

[54875 rows x 8 columns]


C:\Users\someone\AppData\Local\Temp\ipykernel_1300\3309435147.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataframe['area'] = dataframe['area'].replace(reverse_indexes)


Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):

o Ряд VHI для області за вказаний рік;

In [20]:
def vhi_for_area(dataframe, area, year):
    dataframe['Year'] = dataframe['Year'].astype(int)
    filtered_data = dataframe[(dataframe['Year'] == year) & (dataframe['area'] == area)]
    vhi_data = filtered_data[['Year', 'Week', 'VHI', 'area']]
    return vhi_data

print(vhi_for_area(dataframe, 22, 2005))

       Year  Week    VHI  area
23097  2005   1.0  50.17    22
23098  2005   2.0  49.04    22
23099  2005   3.0  48.78    22
23100  2005   4.0  50.65    22
23101  2005   5.0  51.51    22
23102  2005   6.0  52.61    22
23103  2005   7.0  52.88    22
23104  2005   8.0  52.41    22
23105  2005   9.0  53.67    22
23106  2005  10.0  53.33    22
23107  2005  11.0  51.48    22
23108  2005  12.0  49.21    22
23109  2005  13.0  48.57    22
23110  2005  14.0  46.17    22
23111  2005  15.0  42.12    22
23112  2005  16.0  41.62    22
23113  2005  18.0  50.94    22
23114  2005  19.0  54.93    22
23115  2005  20.0  55.55    22
23116  2005  21.0  58.25    22
23117  2005  22.0  60.97    22
23118  2005  23.0  64.44    22
23119  2005  24.0  66.86    22
23120  2005  25.0  68.89    22
23121  2005  26.0  68.79    22
23122  2005  27.0  67.30    22
23123  2005  28.0  65.88    22
23124  2005  29.0  65.73    22
23125  2005  30.0  64.03    22
23126  2005  31.0  63.19    22
23127  2005  32.0  61.87    22
23128  2

o Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани;

In [21]:
def min_max_for_area(dataframe, areas, year):
    for area in areas:
        dataframe['Year'] = dataframe['Year'].astype(int)
        filtered_data = dataframe[(dataframe['Year'] == year) & (dataframe['area'] == area)]
        vhi_data = filtered_data['VHI']

        min_vhi = vhi_data.min()
        max_vhi = vhi_data.max()
        average_vhi = vhi_data.mean()
        median_vhi = vhi_data.median()

        print(f"Область: {new_indexes.get(area)}. Рік: {year}")
        print(f" Мінімальне VHI: {min_vhi}\n Максимальне VHI: {max_vhi}\n Середнє VHI: {average_vhi}\n Медіана VHI: {median_vhi}\n")
    return 0

areas = [2, 3, 22]
min_max_for_area(dataframe, areas, 2005)


Область: Волинська. Рік: 2005
 Мінімальне VHI: 43.14
 Максимальне VHI: 61.8
 Середнє VHI: 53.4186274509804
 Медіана VHI: 53.85

Область: Дніпропетровська. Рік: 2005
 Мінімальне VHI: 29.78
 Максимальне VHI: 72.45
 Середнє VHI: 49.085686274509804
 Медіана VHI: 48.1

Область: Черкаська. Рік: 2005
 Мінімальне VHI: 41.62
 Максимальне VHI: 68.89
 Середнє VHI: 52.22627450980393
 Медіана VHI: 50.65



0

o Ряд VHI за вказаний діапазон років для вказаних областей;

In [22]:
def vhi_for_area_range_years(dataframe, areas, startdate, finaldate):
    dataframe['Year'] = dataframe['Year'].astype(int)
    for area in areas:
        filtered_data = dataframe[(dataframe['Year'] >= startdate) & (dataframe['Year'] <= finaldate) & (dataframe['area'] == area)]
        vhi_data = filtered_data[['Year', 'Week', 'VHI', 'area']]
        print(f"VHI для {new_indexes.get(area)} область за період {startdate}-{finaldate}")
        print(vhi_data)
    return 0

areas = [2, 3, 22]
vhi_for_area_range_years(dataframe, areas, 1990, 2005)

VHI для Волинська область за період 1990-2005
       Year  Week    VHI  area
35525  1990   1.0  36.42     2
35526  1990   2.0  37.35     2
35527  1990   3.0  38.97     2
35528  1990   4.0  42.36     2
35529  1990   5.0  44.08     2
...     ...   ...    ...   ...
36313  2005  48.0  48.73     2
36314  2005  49.0  50.79     2
36315  2005  50.0  50.95     2
36316  2005  51.0  51.57     2
36317  2005  52.0  52.03     2

[793 rows x 4 columns]
VHI для Дніпропетровська область за період 1990-2005
       Year  Week    VHI  area
48695  1990   1.0  40.88     3
48696  1990   2.0  41.08     3
48697  1990   3.0  41.28     3
48698  1990   4.0  41.70     3
48699  1990   5.0  41.75     3
...     ...   ...    ...   ...
49483  2005  48.0  40.70     3
49484  2005  49.0  42.92     3
49485  2005  50.0  41.13     3
49486  2005  51.0  41.49     3
49487  2005  52.0  44.08     3

[793 rows x 4 columns]
VHI для Черкаська область за період 1990-2005
       Year  Week    VHI  area
22355  1990   1.0  33.43    22
2

0

o Для всього набору даних виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25). Повернути роки, назви областей з екстремальними посухами та значення VHI;

In [23]:
def detect_extreme_droughts(dataframe):
    extreme_drought_data = dataframe[dataframe['VHI'] < 15]
    filtered_data = extreme_drought_data[['Year', 'area', 'VHI']]
    vhi_data = filtered_data.groupby(['Year', 'area'])['VHI'].min().reset_index()

    area_counts = vhi_data.groupby('Year')['area'].nunique()
    years_with_enough_areas = area_counts[area_counts >= 5].index

    filtered_vhi_data = vhi_data[vhi_data['Year'].isin(years_with_enough_areas)]
    filtered_vhi_data['area'] = filtered_vhi_data['area'].replace(new_indexes)

    print(filtered_vhi_data)

detect_extreme_droughts(dataframe)


    Year             area    VHI
12  2007       Запорізька  10.88
13  2007     Миколаївська   5.94
14  2007          Одеська   5.52
15  2007       Херсонська  12.23
16  2007  Республіка Крим  13.28


C:\Users\someone\AppData\Local\Temp\ipykernel_1300\3139253269.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_vhi_data['area'] = filtered_vhi_data['area'].replace(new_indexes)
